In [1]:
'''
2023.07.22 토
이 코드파일은 크롤링된 데이터셋을 확인 후 합치고, 그 중 무료인 강의만을 선택해서 데이터셋으로 만드는 코드입니다.
나중에 코드 리팩토링을 해서 유데미의 무료강의에 대해서 처리하는 코드를 추가해 하나로 정리하면 좋을듯합니다!
참고용 카테고리 정보(인덱스 0~9)
categories = ["web-development", "data-science", "mobile-apps", "programming-languages", "game-development",
              "databases", "software-testing", "software-engineering", "development-tools", "no-code-development"]

'''

# 1. 필요 모듈 가져오기
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
import datetime
import time
import pandas as pd
import numpy as np
import pickle as pkl
from tqdm import tqdm

In [29]:
# 1. 3,6 제외한 나머지 것들 먼저 작업함
# df0 = pd.read_csv('../재현/udemy_csv/test_udemy_web-development.csv')
# df2 = pd.read_csv('../찬혁/Crawling/udemy_data/udemy_mobile-apps_230718.csv')
# df1 = pd.read_csv('../찬혁/Crawling/udemy_data/udemy_data-science_230718.csv')
# # df3 = 재현님 수집중
# df4 = pd.read_csv('../재현/udemy_csv/test_udemy_game-development.csv')
# df5 = pd.read_csv('../재현/udemy_csv/test_udemy_databases.csv')
# # df6 = 재현님 확인중
# df7 = pd.read_csv('../인호/02_Crawling/udemy_software-engineering_230720.csv')
# df8 = pd.read_csv('../인호/02_Crawling/udemy_development-tools_230720.csv')
# df9 = pd.read_csv('../인호/02_Crawling/udemy_no-code-development_230720.csv')

# # 새로운 df 만든 후 인덱스 리셋
# new_df = pd.concat([df0, df1, df2, df4, df5, df7, df8, df9], ignore_index=True)

# print(len(new_df))
# new_df.head()

8929


,Unnamed: 0,대분류,소분류,강의명,난이도,가격(현재가격),가격(원래가격),총소요시간,강의소개,언어,출처
0,0.0,web-development,Web Development,【한글자막】 The Web Developer 부트캠프 2023,모든 수준,"₩17,000","₩88,000",73.5시간 주문형 동영상,"배울 내용\n웹 개발 최신 문법 (HTML5, CSS3, 모던 자바스크립트)\nBo...",한국어,https://www.udemy.com/course/the-web-developer...
1,1.0,web-development,Docker,【한글자막】 Docker & Kubernetes : 실전 가이드,모든 수준,"₩17,000","₩88,000",23시간 주문형 동영상,"배울 내용\nDocker와 Kubernetes가 무엇이며, 왜 사용해야 하는지를 배...",한국어,https://www.udemy.com/course/docker-kubernetes...
2,2.0,web-development,Web Development,【한글자막】 100일 코딩 챌린지 - Web Development 부트캠프,모든 수준,"₩17,000","₩88,000",79시간 주문형 동영상,배울 내용\n웹 작동 방식과 웹 개발자로 시작하는 방법\n100일 안에 웹 개발 배...,한국어,https://www.udemy.com/course/100-2022-web-deve...
3,3.0,web-development,React JS,모던 리액트(React)와 리덕스(Redux),모든 수준,"₩17,000","₩88,000",18시간 주문형 동영상,배울 내용\n리액트와 리덕스를 가지고 멋진 싱글 페이지 어플리케이션 만듭니다\n리덕...,한국어,https://www.udemy.com/course/react-redux-korean/
4,4.0,web-development,JavaScript,클린코드 자바스크립트 (2023 업데이트),중급자,"₩17,000","₩88,000",10.5시간 주문형 동영상,배울 내용\n자바스크립트로 코드를 작성하는 방법에 대해 고민해봅니다.\n코드를 작성...,한국어,https://www.udemy.com/course/clean-code-js/


In [2]:
# 2. 3,6 마저 작업
df3 = pd.read_csv('../재현/udemy_csv/한국어,영어강의csv/KE_udemy_programming-languages.csv')
df6 = pd.read_csv('../재현/udemy_csv/udemy_software-testing.csv')

# 새로운 df 만든 후 인덱스 리셋
new_df = pd.concat([df3, df6], ignore_index=True)

print(len(new_df))
new_df.head()

1572


,대분류,소분류,강의명,난이도,가격(현재가격),가격(원래가격),총소요시간,강의소개,언어,출처
0,programming-languages,Python,【한글자막】 Python 부트캠프 : 100개의 프로젝트로 Python 개발 완전 정복,모든 수준,"17,000",88000.0,60시간 주문형 동영상,"배울 내용\nPython 기초 문법\nOOP(객체 지향 프로그래밍), Python ...",한국어,https://www.udemy.com/course/best-100-days-pyt...
1,programming-languages,React JS,"【한글자막】 React 완벽 가이드 with Redux, Next.js, TypeS...",초급자,"17,000",88000.0,48.5시간 주문형 동영상,배울 내용\n리액티브 웹 앱의 성능을 향상시키는 법\nJavaScript의 장점을 ...,한국어,https://www.udemy.com/course/best-react/
2,programming-languages,Java Algorithms,【한글자막】 JavaScript 알고리즘 & 자료구조 마스터클래스,모든 수준,"17,000",88000.0,21.5시간 주문형 동영상,배울 내용\nBig O 표기법을 기반으로한 알고리즘 성능 평가법\n재귀(Recurs...,한국어,https://www.udemy.com/course/best-javascript-d...
3,programming-languages,C++ (programming language),C++ 언매니지드 프로그래밍,중급자,"219,000",NaN,25.5시간 주문형 동영상,배울 내용\nC++ 언어 사용법\n올바른 프로그래밍 습관\nC++ 언어가 내부적으로...,한국어,https://www.udemy.com/course/cpp-unmanaged-pro...
4,programming-languages,JavaScript,【한글자막】 JavaScript 완벽 가이드 : 초급 + 고급 마스터 과정,모든 수준,"17,000",88000.0,50.5시간 주문형 동영상,배울 내용\n처음부터 시작하는 JavaScript - 초급부터 고급까지\n최신 Ja...,한국어,https://www.udemy.com/course/javascript-zw/


In [3]:
# 가격(현재가격)이 "무료"인것만
free_df = new_df[new_df["가격(현재가격)"]=="무료"]
print(len(free_df), "개의 무료 강의가 있습니다")

153 개의 무료 강의가 있습니다


In [4]:
# free_df = free_df.drop(columns=['Unnamed: 0']) #Unnamed: 0 col이 있는 경우 제외
free_df = free_df.reset_index(drop=True)
free_df.head()

,대분류,소분류,강의명,난이도,가격(현재가격),가격(원래가격),총소요시간,강의소개,언어,출처
0,programming-languages,Python,김왼손의 유기농냠냠파이썬: 세시간만에 끝내는 파이썬 (Python) 기초,초급자,무료,NaN,NaN,NaN,한국어,https://www.udemy.com/course/unp_python/
1,programming-languages,C (programming language),C언어 기초 프로그래밍 강좌 (C Programming Tutorial),초급자,무료,NaN,NaN,NaN,한국어,https://www.udemy.com/course/c-programming-basic/
2,programming-languages,C (programming language),C 프로그래밍 - 입문부터 게임 개발까지,초급자,무료,NaN,NaN,NaN,한국어,https://www.udemy.com/course/nadocoding-c/
3,programming-languages,JavaScript,[생활코딩] WEB - 자바스크립트(JavaScript) 기초 #1,초급자,무료,NaN,NaN,NaN,한국어,https://www.udemy.com/course/web-javascript-1/
4,programming-languages,Java,[생활코딩] 자바(JAVA) 입문 #1,초급자,무료,NaN,NaN,NaN,한국어,https://www.udemy.com/course/java-1-d/


In [5]:
# 태그가 없는 경우, nan값을 할당하고 넘어가는 함수를 미리 정의합니다.
def find_element_nan(driver, path):
    try:
        element = driver.find_element(By.XPATH, path)
        return element.text
    except:
        return np.nan

In [6]:
# 🌟 중간에 코드가 끊겨서 다시 재시작한다면 restart = True 로 변경
restart = False

# 1. 실행시간 측정
start_time = time.time()


# 2. free_current_info를 저장할 pkl 파일 정보 확인
free_current_info = {"idx": 0} 

if restart == True :
    free_df = pd.read_csv(f"./udemy_free_36_230723.csv") 
    with open('free_current_info.pickle', 'rb') as file:
        free_current_info = pkl.load(file)
    idx = free_current_info['idx']

idx = free_current_info['idx']

# 3. 크롤링 시작
print(f"크롤링 시작 : 총 {len(free_df)}페이지 중 {idx}번째 강의부터 수집 시작 ")

try:
    options = uc.ChromeOptions()
    driver = uc.Chrome(use_subprocess=True, options=options)
    for index, row in tqdm(free_df.iloc[idx:].iterrows()):
        # 4. pickle 파일 저장
        free_current_info["idx"] = index
        with open('free_current_info.pickle', 'wb') as file: 
            pkl.dump(free_current_info, file)
        # 5. row를 하나씩 돌면서 ['출처']에 접속한 후 총소요시간, 강의소개 채우기!
        url = row['출처']
        while True:
            try:     
                driver.get(url)
                break
            except Exception as e:
                exception_name = type(e).__name__
                print(f"🥲 네트워크가 연결이 잘 안돼요... Exception: {exception_name}")
                time.sleep(3)
        driver.maximize_window()
        time.sleep(3)
        free_df.at[index, '총소요시간'] = find_element_nan(driver, '//div[@class="course-content-length--course-content-length--zNAIv"]')
        free_df.at[index, '강의소개'] = find_element_nan(driver, '//div[@data-purpose="content-container-wrapper"]')
        free_df.to_csv(f"./udemy_free_36_230723.csv", index=False)
    time.sleep(3)
    driver.quit()    
    print(f"👍 크롤링 완료 : 총 {len(free_df)}페이지 수집 완료")

except KeyboardInterrupt:
    print(f"👍 크롤링 완료 : 총 {len(free_df)}페이지 중 {idx}번째 강의까지 수집 완료")

크롤링 시작 : 총 153페이지 중 0번째 강의부터 수집 시작 


153it [11:01,  4.33s/it]


👍 크롤링 완료 : 총 153페이지 수집 완료


In [34]:
# # test!!
# free_df_sample = free_df[:3]

# options = uc.ChromeOptions()
# driver = uc.Chrome(use_subprocess=True, options=options)
# idx=1
# for index, row in free_df_sample.iloc[idx:].iterrows():
#     url = row['출처']
#     driver.get(url)
#     free_df_sample.at[index, '총소요시간'] = find_element_nan(driver, '//div[@class="course-content-length--course-content-length--zNAIv"]')
#     free_df_sample.at[index, '강의소개'] = find_element_nan(driver, '//div[@data-purpose="content-container-wrapper"]')
# time.sleep(3)
# driver.quit

# print(free_df_sample)

               대분류     소분류                                         강의명  난이도  \
0  web-development    HTML          [생활코딩] WEB - HTML & Internet 기초 #1  초급자   
1  web-development  Docker                        [생활코딩] 도커(Docker) 기초  초급자   
2  web-development  Django  [생활코딩] WEB - Python - Django Web Framework  초급자   

  가격(현재가격) 가격(원래가격)            총소요시간  \
0       무료      NaN              NaN   
1       무료      NaN     온디맨드 동영상 42분   
2       무료      NaN  온디맨드 동영상 1시간 9분   

                                                강의소개   언어  \
0                                                NaN  한국어   
1       도커 기본개념 & 설치하기\n도커 이미지pull\n컨테이너run\n도커 네트워크  한국어   
2  Django(장고) 설치 방법\nDjango(장고) 기본 개념\nDjango(장고)...  한국어   

                                                  출처  
0  https://www.udemy.com/course/web-html-internet-1/  
1            https://www.udemy.com/course/docker-ky/  
2  https://www.udemy.com/course/web-python-django...  


In [9]:
# 3. 파일을 합치고, 한국어/영어인 것만도 따로 빼서 저장합니다.
df_36 = pd.read_csv('./udemy_free_36_230723.csv')
df_etc = pd.read_csv('./udemy_free_230722.csv')

# 새로운 df 만든 후 인덱스 리셋
df_all = pd.concat([df_36, df_etc], ignore_index=True)
df_all


,대분류,소분류,강의명,난이도,가격(현재가격),가격(원래가격),총소요시간,강의소개,언어,출처
0,programming-languages,Python,김왼손의 유기농냠냠파이썬: 세시간만에 끝내는 파이썬 (Python) 기초,초급자,무료,NaN,온디맨드 동영상 2시간 58분,"파이썬의 기초를 쉽게 배울 수 있습니다.\n파이썬이 뭔지, 프로그래밍이 뭔지 감을 ...",한국어,https://www.udemy.com/course/unp_python/
1,programming-languages,C (programming language),C언어 기초 프로그래밍 강좌 (C Programming Tutorial),초급자,무료,NaN,온디맨드 동영상 4시간 21분,C언어의 기본 이론과 기초 프로그래밍,한국어,https://www.udemy.com/course/c-programming-basic/
2,programming-languages,C (programming language),C 프로그래밍 - 입문부터 게임 개발까지,초급자,무료,NaN,온디맨드 동영상 6시간 36분,C 프로그래밍 기초 마스터 + 9가지 실전 게임을 제작합니다. 언어를 배우는 것은 ...,한국어,https://www.udemy.com/course/nadocoding-c/
3,programming-languages,JavaScript,[생활코딩] WEB - 자바스크립트(JavaScript) 기초 #1,초급자,무료,NaN,온디맨드 동영상 1시간 26분,Javascript 수업의 목적\nHTML 태그와 Javascript\nCSS 선택...,한국어,https://www.udemy.com/course/web-javascript-1/
4,programming-languages,Java,[생활코딩] 자바(JAVA) 입문 #1,초급자,무료,NaN,온디맨드 동영상 1시간 56분,Java 입문 A to Z\nJava 설치하는 방법\nJava 실행\nJava 기술...,한국어,https://www.udemy.com/course/java-1-d/
...,...,...,...,...,...,...,...,...,...,...
1196,no-code-development,WordPress,WordPressサイト最強のGenesisFrameworkを無料でセットアップする方法【...,모든 수준,무료,NaN,온디맨드 동영상 1시간 1분,SEOとセキュリティに強いWordPressサイトを無料で作れます\nシンプルかつ洗練された...,일본어,https://www.udemy.com/course/genesisframework2...
1197,no-code-development,Software Development,codeorg Temel Eğitim Dersleri Kursu,초급자,무료,NaN,온디맨드 동영상 1시간 3분,Temel Kod Blokları\nProgramlamaya Giriş Eğitimi,터키어,https://www.udemy.com/course/codeorg-temel-egi...
1198,no-code-development,WordPress,Создание сайта без знаний веб-программирования,초급자,무료,NaN,온디맨드 동영상 1시간 55분,Вы узнаете как самому сделать сайт с нуля без ...,러시아어,https://www.udemy.com/course/how-to-create-web...
1199,no-code-development,Landing Page Optimization,flexifunnel(फ्लेक्सी फनल) ka use krke landing ...,초급자,무료,NaN,온디맨드 동영상 1시간 3분,इस कोर्स को पूरा करने के पश्चात आप अपने क्लाइं...,Hindi,https://www.udemy.com/course/flexifunnel-tutor...


In [11]:
#총소요시간, 강의소개 null값 4개 -> 없어진 강의, 무료에서 유료가 된 강의인데 언어가 이상한 거라 삭제처리
df_all.isna().sum()

대분류            0
소분류            8
강의명            4
난이도            0
가격(현재가격)       0
가격(원래가격)    1201
총소요시간          4
강의소개           4
언어             4
출처             0
dtype: int64

In [31]:
df_all = df_all.dropna(subset=['총소요시간'])
df_all.isna().sum()

대분류            0
소분류            6
강의명            2
난이도            0
가격(현재가격)       0
가격(원래가격)    1197
총소요시간          0
강의소개           0
언어             2
출처             0
dtype: int64

In [34]:
df_all = df_all.dropna(subset=['소분류'])
df_all.isna().sum()

대분류            0
소분류            0
강의명            0
난이도            0
가격(현재가격)       0
가격(원래가격)    1191
총소요시간          0
강의소개           0
언어             0
출처             0
dtype: int64

In [35]:
df_all['언어'].unique()

array(['한국어', '영어', '포르투갈어', '스페인어', 'Indonesian', '러시아어', '루마니아어',
       'Arabic', 'Vietnamese', 'Traditional Chinese', '독일어', '터키어',
       '폴란드어', '프랑스어', 'Bengali', '일본어', '이탈리아어', 'Telugu', 'Thai',
       'Azeri', 'Tamil', 'Urdu', 'Hindi', 'Greek', 'Simplified Chinese',
       'Afrikaans', 'Marathi', 'Kannada'], dtype=object)

In [38]:
# 언어 전체, 한국어or영어(KE)로 따로 저장합니다.
df_all.to_csv("./all_udemy_free.csv", index=False)

In [39]:
df_ke = df_all[df_all['언어'].isin(['한국어', '영어'])]
df_ke['언어'].unique()

array(['한국어', '영어'], dtype=object)

In [40]:
df_ke.to_csv("./ke_udemy_free.csv", index=False)

In [41]:
print(len(df_all))
print(len(df_ke))

1191
584
